In [11]:
# http://www.laizquierdadiario.com/spip.php?page=backend_portada
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_laizquierdadiario_article(html):
    article = {
        'section': '',
        'volanta': '',
        'headline': '',
        'bajada': '',
        'articleBody': ''
    }
    
    soup = BeautifulSoup(html, 'html.parser')
        
    header = soup.find('div', {'class': 'category-wrapper'})
    if header:
        # section
        section = header.find('a', {'class': 'category'})
        if section:          
            article['section'] = section.text.strip()
            
        # volanta
        volanta = header.find('a', {'class': 'category grouper'})
        if volanta:          
            article['volanta'] = volanta.text.strip()

    # headline
    headline = soup.find('h1', {'class': 'title'})
    if headline:
        article['headline'] = headline.text.strip()
        
    # bajada
    bajada = soup.find('p', {'class': 'preview'})
    if bajada:
        article['bajada'] = bajada.text.strip()
    
    # articleBody
    articleBody = soup.find('section', {'class': 'note-body'})
    if articleBody:
        article['articleBody'] = articleBody.text.strip()
    
    return article



headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

links = []

xml_request = requests.get('http://www.laizquierdadiario.com/spip.php?page=backend_portadal', headers)

print('grabbing La Izquierda Diario links')
print(xml_request.status_code)
if xml_request.status_code == 200:
    soup = BeautifulSoup(xml_request.content, 'html.parser')
    items = soup.find_all('item')
    print(items)
    for item in items:
        link = item.find('link')
        links.append(link.text)

print(len(links), 'obtained')

grabbing La Izquierda Diario links
0 obtained


In [9]:
links[50:100]

['http://www.laizquierdadiario.com/CULTURA-9883',
 'http://www.laizquierdadiario.com/05-10-2017',
 'http://www.laizquierdadiario.com/CULTURA-9976',
 'http://www.laizquierdadiario.com/07-10-2017',
 'http://www.laizquierdadiario.com/CULTURA-9996',
 'http://www.laizquierdadiario.com/ECONOMIA-10029',
 'http://www.laizquierdadiario.com/11-10-2014',
 'http://www.laizquierdadiario.com/2014-3308',
 'http://www.laizquierdadiario.com/rubrique3311',
 'http://www.laizquierdadiario.com/Cultura-470',
 'http://www.laizquierdadiario.com/11-10-2017',
 'http://www.laizquierdadiario.com/POLITICA-10013',
 'http://www.laizquierdadiario.com/SOCIEDAD-10017',
 'http://www.laizquierdadiario.com/Aca-nos-podes-encontrar',
 'http://www.laizquierdadiario.com/Cordoba',
 'http://www.laizquierdadiario.com/Economia',
 'http://www.laizquierdadiario.com/CULTURA-10016',
 'http://www.laizquierdadiario.com/10-10-2017',
 'http://www.laizquierdadiario.com/GENEROS-Y-SEXUALIDADES-10007',
 'http://www.laizquierdadiario.com/DEPO

In [ ]:


articles = []

for link in links:
    print('scrapping:', link)
    article_request = requests.get(link, headers)
    
    if article_request.status_code == 200:  
        article = parse_laizquierdadiario_article(article_request.content)
        article['link'] = link
        article['source'] = 'laizquierdadiario'
        articles.append(article)
        df = pd.DataFrame(articles)
        df.to_csv('laizquierdadiario.csv', index=False)